tool_list =  
(
    [add, multiply, search_tool, retriever_tool]  
     + gmail_tool_list + loaded_tool_list
)

* from langgraph.checkpoint.memory import MemorySaver
* DuckDuckGoSearchRun
* from langchain.agents import load_tools
* create_retriever_tool
* ToolNode

### tools 선언

In [70]:
from dotenv import load_dotenv

load_dotenv()

True

In [71]:
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """숫자 a와 b를 더합니다."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """숫자 a와 b를 곱합니다."""
    return a * b

In [72]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

In [73]:
from langchain_google_community import GmailToolkit

from langchain_google_community.gmail.utils import (
    build_resource_service,
    get_gmail_credentials,
)

# Can review scopes here https://developers.google.com/gmail/api/auth/scopes
# For instance, readonly scope is 'https://www.googleapis.com/auth/gmail.readonly'
credentials = get_gmail_credentials(
    token_file="google/token.json",
    scopes=["https://mail.google.com/"],
    client_secrets_file="google/credentials.json",
)
api_resource = build_resource_service(credentials=credentials)
gmail_toolkit = GmailToolkit(api_resource=api_resource)
gmail_tool_list = gmail_toolkit.get_tools()

In [74]:
from langchain.agents import load_tools

loaded_tool_list = load_tools(
    ["arxiv"],
)

In [75]:
import os

from langchain_pinecone import PineconeVectorStore
from langchain_upstage import UpstageEmbeddings
from langchain_core.tools.retriever import create_retriever_tool

embedding_function = UpstageEmbeddings(
    model="embedding-query"
)
vector_store = PineconeVectorStore(
    embedding=embedding_function,
    index_name="real-estate",
)
retriever = vector_store.as_retriever(search_kwargs={"k": 3})
retriever_tool = create_retriever_tool(
    retriever=retriever,
    name="real_estate_tax_retriever",
    description="Contains information about real estate tax up to December 2024",
)

In [76]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

In [77]:
from langgraph.prebuilt import ToolNode

tool_list = [retriever_tool, search_tool, add, multiply] + loaded_tool_list + gmail_tool_list

llm_with_tools = llm.bind_tools(tool_list)
tool_node = ToolNode(tool_list)

### state 정의하기

In [79]:
from langgraph.graph import StateGraph, MessagesState

class AgentState(MessagesState):
    summary: str

graph_builder = StateGraph(AgentState)

### Node 만들기

In [80]:
from langchain_core.messages import SystemMessage

def agent(state: AgentState):
    
    messages = state["messages"]
    summary = state["summary"]

    if summary != "":
        messages = [SystemMessage(content=f"""
        이것은 이전 대화들을 요약한 것입니다. 참고하여주세요.\n요약: {summary}
        """)] + messages

    response = llm_with_tools.invoke(messages)

    return {"messages": [response]}

In [84]:
from langchain_core.messages import HumanMessage

query = "내 메일로 최신 ai 논문을 요약해서 발송해줘. 내 메일: wogusto13@gmal.com"

initial_state = {"messages": [HumanMessage(query)], "summary": ""}

In [86]:
response = agent(initial_state)

In [88]:
tool_response = tool_node.invoke(response)

In [97]:
tool_response['messages'][-1].tool_calls

AttributeError: 'ToolMessage' object has no attribute 'tool_calls'

In [ ]:
from langgraph.types import interrupt, Command

def human_review(state: AgentState):

    messages = state["messages"]
    last_message = messages[-1]
    tool_call = last_message.tool_calls[-1]
    
    human_review = interrupt({"question": "이렇게 진행하면 될까요?", "tool_message": tool_call})
    review_action = human_review["action"]

    review_data = human_review.get("data", None)

    if review_action == "continue":
        return Command(goto="tools")
    
    if review_action == "update_args": # 도구는 맞게 골랐는데 args가 마음에 안드는 경우

        update_ai_message = {
            'id': last_message.id,
            'role': "ai",
            "content": last_message.content,
            "tool_calls": [
                {"id": tool_call["id"], "name": tool_call["name"], "args": review_data}
            ]
        }
        return Command(goto="tools", update={"messages": [update_ai_message]})

    if review_action == "update_tool": # tool_message가 마음에 안드는 경우

        update_tool_message = {
            "name": 


        }


    

In [ ]:
def summarize_messages(state: AgentState):

    messages = state["messages"]
    summary = state["summary"]

    summarize_prompt = f"이 대화들을 요약해주세요. 대화 리스트: {messages}"

    if summary != "":
        summarize_prompt = f"""
        이전에 요약된 내용을 포함해서 아래의 대화들을 요약해주세요.
        요약: {summary},
        대화: {messages}
        """

    response = llm.invoke(summarize_prompt)
    
    return {"summary": response.content}

In [ ]:
from langchain_core.messages import RemoveMessage

def delete_messages(state:AgentState):

    messages = state["messages"]

    deleted_messages = [RemoveMessage(id=message.id) for message in messages[:-3]]

    return {"messages": deleted_messages}


In [ ]:
def should_continue(state: AgentState):

    messages = state["messages"]

    if messages[-1].tool_calls:
        return "human_review"
        
    return "summarize_messages"